In [ ]:
import polars as pl
from datetime import timedelta
from pathlib import Path
from tqdm import tqdm
directory = "../data/stocks"



In [ ]:

returns = {
    "yearly": {},
    "monthly": {},
    "weekly": {},
    "daily": {},
}

all_dates = set()
columns = set()
errors = []
for file in tqdm(sorted(Path(directory).glob("*.csv")), desc="Processing Dates"):
    df = pl.read_csv(file)
    try:
        df = pl.read_csv(file)
        df = df.filter((pl.col("Open") > 0) & (pl.col("Close") > 0))
    except Exception as e:
        errors.append((file, e))
        continue
    columns.add(file.stem)
    all_dates.update(df["Date"])
all_dates = sorted(all_dates)
columns = sorted(columns)
for error in errors:
    print(error)
data = {
    "Date": all_dates,
    # **{column: [None] * len(all_dates) for column in columns}
}
all_df = pl.DataFrame(
    data
)


In [ ]:
errors = []
for file in tqdm(sorted(Path(directory).glob("*.csv")), desc="Processing Returns"):
    df = pl.read_csv(file)
    try:
        df = pl.read_csv(file)
        df = df.filter((pl.col("Open") > 0) & (pl.col("Close") > 0))
    except Exception as e:
        errors.append((file, e))
        continue
    df = df.with_columns((pl.col("Close") / pl.col("Open")).alias(file.stem))
    df = df.select(["Date", file.stem])
    all_df = all_df.join(df, on="Date", how="outer")

for error in errors:
    print(error)


In [ ]:
ticker_counts = all_df.apply(lambda row: sum(1 for value in row if value))
all_df = all_df.with_columns(ticker_counts.rename({'apply': 'ticker_count'}))
all_df = all_df.with_columns(pl.all().fill_null(strategy="zero"))
all_df = all_df.with_columns(pl.sum_horizontal(*columns).alias("sum"))
all_df = all_df.with_columns((pl.col("sum") / pl.col('ticker_count')).alias("avg"))
all_df.select('avg').write_csv("../data/results/aligned.csv")



In [ ]:
all_df = pl.DataFrame(
    data
)
errors = []
for file in tqdm(sorted(Path(directory).glob("*.csv")), desc="Processing Returns"):
    df = pl.read_csv(file)
    try:
        df = pl.read_csv(file)
        df = df.filter((pl.col("Open") > 0) & (pl.col("Close") > 0))
    except Exception as e:
        errors.append((file, e))
        continue
    df = df.with_columns((pl.col("Close")).alias(file.stem))
    df = df.select(["Date", file.stem])
    all_df = all_df.join(df, on="Date", how="outer")

for error in errors:
    print(error)
    
ticker_counts = all_df.apply(lambda row: sum(1 for value in row if value))
all_df = all_df.with_columns(ticker_counts.rename({'apply': 'ticker_count'}))
all_df = all_df.with_columns(pl.all().fill_null(strategy="zero"))

In [ ]:
truncated_df = all_df.filter(pl.col("ticker_count") > 100)
truncated_df

In [ ]:

truncated_df = truncated_df.with_columns(pl.col("Date").str.to_date("%Y-%m-%d"))
current_y = truncated_df[0]
current_m = truncated_df[0]
keep_y = []
keep_m = []
for i in tqdm(range(truncated_df.shape[0]), desc="Processing rows"):
    row = truncated_df[i]
    # year check
    if row["Date"][0] - current_y["Date"][0] > timedelta(days=365):
        keep_y.append(i)
        current_y = row
    # month check
    if row["Date"][0] - current_m["Date"][0] > timedelta(days=30):
        keep_m.append(i)
        current_m = row
        


In [ ]:
monthly_df = all_df[keep_m]
yearly_df = all_df[keep_y]

In [ ]:
monthly_returns = monthly_df.select(pl.col(columns))
monthly_returns

In [ ]:
yearly_returns = yearly_df.select(pl.col(columns))
yearly_returns